## Step 1: Install Dependencies


In [1]:
# Install dependencies 
import sys

print("Checking dependencies...")

try:
    import sqlalchemy
    import pandas as pd
    import yaml
    print("All dependencies installed!")
    print(f"   SQLAlchemy: {sqlalchemy.__version__}")
    print(f"   Pandas: {pd.__version__}")
except ImportError as e:
    print(f"Installing missing dependencies: {e}")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "sqlalchemy", "pandas", "pyyaml"])
    print("Installation complete! Please restart kernel and re-run.")


Checking dependencies...
All dependencies installed!
   SQLAlchemy: 2.0.43
   Pandas: 2.3.3


## Step 2: Import Modules


In [2]:
# Import all required modules
import pandas as pd
import yaml
from datetime import datetime, timedelta
from pathlib import Path

# Scheduler modules
from scheduler.domain.db import init_database, get_session
from scheduler.domain.models import Employee, Shift, Assignment
from scheduler.domain.repositories import (
    EmployeeRepository, ShiftRepository, AssignmentRepository
)
from scheduler.io.import_csv import import_employees_csv, import_shifts_csv
from scheduler.io.export_csv import export_assignments_csv
from scheduler.io.config import load_config
from scheduler.engine.orchestrator import build_week_schedule

print("All modules imported successfully!")
print("v2.1 Architecture:")
print("   • Role-specific schedulers (Manager, Sandwich, Cohort)")
print("   • SQLite database with SQLAlchemy ORM")
print("   • Orchestrator pattern for coordination")
print("   • Enhanced fairness distribution")


All modules imported successfully!
v2.1 Architecture:
   • Role-specific schedulers (Manager, Sandwich, Cohort)
   • SQLite database with SQLAlchemy ORM
   • Orchestrator pattern for coordination
   • Enhanced fairness distribution


## Step 3: Configuration


In [3]:
# Configuration
DB_URL = "sqlite:///scheduler_present.db"
WEEK_ID = "2025-W48"  # Week to schedule
OUTPUT_CSV = "schedule_present_2025_week_48.csv"
CONFIG_PATH = "scheduler_config.yaml"

# CSV file paths
EMPLOYEES_CSV = "data/employees_new_12w_v2.csv"
SHIFTS_CSV = "data/shift_new_12w_v2.csv"

print("Configuration:")
print(f"   Database: {DB_URL}")
print(f"   Week: {WEEK_ID}")
print(f"   Output: {OUTPUT_CSV}")
print(f"   Config: {CONFIG_PATH}")
print()
print("Ready to start!")


Configuration:
   Database: sqlite:///scheduler_present.db
   Week: 2025-W48
   Output: schedule_present_2025_week_48.csv
   Config: scheduler_config.yaml

Ready to start!


## Step 4: Initialize Database


In [4]:
print("Initializing database...")

# Remove old database if exists
db_path = Path(DB_URL.replace("sqlite:///", ""))
if db_path.exists():
    print(f"Removing old database: {db_path}")
    db_path.unlink()

# Create engine and initialize database
from scheduler.domain.db import create_db_engine
from scheduler.domain.models import Base

engine = create_db_engine(DB_URL)
print(f"Engine created for: {DB_URL}")

# Create all tables
Base.metadata.create_all(engine)
print("All tables created")

# Create session using the same engine
from sqlalchemy.orm import sessionmaker
SessionLocal = sessionmaker(bind=engine)
session = SessionLocal()
print("Session ready")


Initializing database...
Removing old database: scheduler_present.db
Engine created for: sqlite:///scheduler_present.db
All tables created
Session ready


In [6]:
# Verify tables were created
print("Verifying database tables...")
from sqlalchemy import inspect

inspector = inspect(engine)
tables = inspector.get_table_names()
print(f"Created tables: {tables}")

if 'employees' in tables and 'shifts' in tables and 'assignments' in tables:
    print("All required tables created successfully!")
else:
    print("ERROR: Missing required tables!")
    print("Expected: employees, shifts, assignments")
    print(f"Found: {tables}")
    
    # Try to create tables again
    print("Attempting to create tables again...")
    Base.metadata.create_all(engine)
    
    # Check again
    inspector = inspect(engine)
    tables = inspector.get_table_names()
    print(f"Tables after retry: {tables}")


Verifying database tables...
Created tables: ['assignments', 'employees', 'feedback', 'shifts']
All required tables created successfully!


## Step 5: Import Data


In [11]:
print("Importing data from CSV...")

# Clear existing data to avoid conflicts
print("Clearing existing data...")

# Rollback any pending transactions first
session.rollback()
print("   Rolled back pending transactions")

# Import text function for raw SQL
from sqlalchemy import text

# Delete existing data in correct order (respecting foreign key constraints)
session.execute(text("DELETE FROM assignments"))
print("   Cleared assignments")

session.execute(text("DELETE FROM shifts"))
print("   Cleared shifts")

session.execute(text("DELETE FROM employees"))
print("   Cleared employees")

session.commit()
print("   Database cleared successfully")

# Import employees
employee_count = import_employees_csv(session, EMPLOYEES_CSV)
print(f"Imported {employee_count} employees")

# Import shifts
shift_count = import_shifts_csv(session, SHIFTS_CSV)
print(f"Imported {shift_count} total shifts")

print(f"\nImport complete!")
print(f"   Employees: {employee_count}")
print(f"   Total shifts: {shift_count}")

# Get employees from database to display
from scheduler.domain.repositories import EmployeeRepository
employees = EmployeeRepository.get_all(session)

print(f"\nEmployees:")
for emp in employees:
    print(f"   {emp.first_name} {emp.last_name} ({emp.primary_role})")


Importing data from CSV...
Clearing existing data...
   Rolled back pending transactions
   Cleared assignments
   Cleared shifts
   Cleared employees
   Database cleared successfully
[INFO] Imported 8 employees from data/employees_new_12w_v2.csv
Imported 8 employees
[INFO] Imported 84 shifts from data/shift_new_12w_v2.csv
Imported 84 total shifts

Import complete!
   Employees: 8
   Total shifts: 84

Employees:
   Max Hayes (MANAGER)
   Mia Stone (MANAGER)
   Wendy Ng (WAITER)
   Will Brown (WAITER)
   Bella Tran (BARISTA)
   Ben Park (BARISTA)
   Sam Lee (SANDWICH)
   Sara Khan (SANDWICH)


## Step 6: Load Configuration and Check Shifts


In [12]:
print("Loading configuration...")

# Load configuration
cfg = load_config(CONFIG_PATH)

print(f"Configuration loaded!")
print(f"   Timezone: {cfg.timezone}")
print(f"   Fairness penalty: {cfg.weights.fairness_penalty_per_std_above_median}")
print(f"   Weekend requirements: {cfg.weekend_requirements}")

# Check if we have shifts for this week
print(f"\nChecking shifts for {WEEK_ID}...")
week_shifts = ShiftRepository.get_by_week(session, WEEK_ID)

if not week_shifts:
    print(f"   Creating synthetic shifts for {WEEK_ID}...")
    
    # Create synthetic shifts for the week
    from datetime import datetime, timedelta
    
    # Parse week ID to get start date
    year, week = WEEK_ID.split('-W')
    year, week = int(year), int(week)
    
    # Get Monday of the week
    jan_1 = datetime(year, 1, 1)
    days_since_jan_1 = (week - 1) * 7
    monday = jan_1 + timedelta(days=days_since_jan_1)
    
    # Create shifts for each day of the week
    for i in range(7):
        shift_date = monday + timedelta(days=i)
        shift = Shift(
            date=shift_date.date(),
            week_id=WEEK_ID
        )
        session.add(shift)
    
    session.commit()
    week_shifts = ShiftRepository.get_by_week(session, WEEK_ID)
    print(f"   Created {len(week_shifts)} shifts")
else:
    print(f"   Found {len(week_shifts)} existing shifts")

print(f"\nShifts for {WEEK_ID}:")
for shift in week_shifts:
    day = shift.date.strftime("%A")
    print(f"   {shift.date} ({day})")


Loading configuration...
Configuration loaded!
   Timezone: Australia/Sydney
   Fairness penalty: 2.0
   Weekend requirements: {'MANAGER': 2, 'BARISTA': 1, 'WAITER': 2, 'SANDWICH': 1}

Checking shifts for 2025-W48...
   Creating synthetic shifts for 2025-W48...
   Created 7 shifts

Shifts for 2025-W48:
   2025-11-26 (Wednesday)
   2025-11-27 (Thursday)
   2025-11-28 (Friday)
   2025-11-29 (Saturday)
   2025-11-30 (Sunday)
   2025-12-01 (Monday)
   2025-12-02 (Tuesday)


## Step 7: Generate Schedule


In [13]:
print("Generating schedule with v2.1 architecture...")
print("=" * 60)
print("Orchestrator will run:")
print("   1. ManagerScheduler (weekend-first)")
print("   2. SandwichScheduler (early morning)")
print("   3. CohortScheduler(BARISTA) (fair distribution)")
print("   4. CohortScheduler(WAITER) (fair distribution)")
print()

try:
    # Build schedule using orchestrator
    assignments = build_week_schedule(
        session=session,
        week_id=WEEK_ID,
        cfg=cfg,
        scheduler_order=["MANAGER", "SANDWICH", "BARISTA", "WAITER"],
        persist=True  # Save to database
    )
    
    print(f"\nSUCCESS! Schedule generated!")
    print(f"   Total assignments: {len(assignments)}")
    
    # Show role distribution
    role_counts = {}
    for assignment in assignments:
        role = assignment.role
        role_counts[role] = role_counts.get(role, 0) + 1
    
    print(f"\nBy Role:")
    for role, count in role_counts.items():
        print(f"   {role}: {count} assignments")
        
except Exception as e:
    print(f"ERROR: {e}")
    assignments = []


Generating schedule with v2.1 architecture...
Orchestrator will run:
   1. ManagerScheduler (weekend-first)
   2. SandwichScheduler (early morning)
   3. CohortScheduler(BARISTA) (fair distribution)
   4. CohortScheduler(WAITER) (fair distribution)

[INFO] Orchestrator: Building schedule for 2025-W48
[INFO] Scheduler order: ['MANAGER', 'SANDWICH', 'BARISTA', 'WAITER']

[INFO] Running MANAGER scheduler...
[INFO] ManagerScheduler: Generated 9 assignments
[OK] MANAGER scheduler completed: 9 assignments

[INFO] Running SANDWICH scheduler...
[INFO] SandwichScheduler: Generated 7 assignments
[OK] SANDWICH scheduler completed: 7 assignments

[INFO] Running BARISTA scheduler...
[INFO] BARISTAScheduler: Generated 7 assignments
[OK] BARISTA scheduler completed: 7 assignments

[INFO] Running WAITER scheduler...
[INFO] WAITERScheduler: Generated 9 assignments
[OK] WAITER scheduler completed: 9 assignments

[INFO] Validating complete schedule...
[OK] All assignment constraints validated
[OK] Orches

## Step 8: Detailed Scheduling Logs


In [14]:
print("DETAILED SCHEDULING LOGS")
print("=" * 80)

# Get all assignments from database for detailed analysis
week_assignments = AssignmentRepository.get_by_week(session, WEEK_ID)
print(f"\nWeek {WEEK_ID} - {len(week_shifts)} shifts, {len(week_assignments)} assignments")

# Group by role for detailed analysis
role_assignments = {}
for assignment in week_assignments:
    role = assignment.role
    if role not in role_assignments:
        role_assignments[role] = []
    role_assignments[role].append(assignment)

print(f"\nAssignments by Role:")
for role, assigns in role_assignments.items():
    print(f"\n{role} ROLE ({len(assigns)} assignments):")
    
    # Group by employee
    emp_assignments = {}
    for assign in assigns:
        emp_id = assign.emp_id
        if emp_id not in emp_assignments:
            emp_assignments[emp_id] = []
        emp_assignments[emp_id].append(assign)
    
    # Show each employee's schedule
    for emp_id, emp_assigns in emp_assignments.items():
        employee = EmployeeRepository.get_by_id(session, emp_id)
        
        # Calculate total hours from start_time and end_time
        total_hours = 0
        for assign in emp_assigns:
            duration = assign.end_time - assign.start_time
            total_hours += duration.total_seconds() / 3600  # Convert to hours
        
        print(f"\n   {employee.first_name} {employee.last_name} ({employee.primary_role})")
        print(f"      Total: {total_hours:.1f}h across {len(emp_assigns)} shifts")
        print(f"      Skills: Coffee={employee.skill_coffee}, Sandwich={employee.skill_sandwich}, Service={employee.customer_service_rating}, Speed={employee.skill_speed}")
        
        # Show each shift
        for assign in sorted(emp_assigns, key=lambda x: x.start_time):
            shift = ShiftRepository.get_by_id(session, assign.shift_id)
            day_name = shift.date.strftime("%A")
            start_time = assign.start_time.strftime("%H:%M")
            end_time = assign.end_time.strftime("%H:%M")
            
            # Calculate shift hours
            duration = assign.end_time - assign.start_time
            hours = duration.total_seconds() / 3600
            
            print(f"      {shift.date} ({day_name}): {start_time}-{end_time} ({hours:.1f}h)")

# Show fairness analysis
print(f"\nFAIRNESS ANALYSIS:")
print("=" * 50)

# Calculate hours per employee
emp_hours = {}
for assignment in week_assignments:
    emp_id = assignment.emp_id
    if emp_id not in emp_hours:
        emp_hours[emp_id] = 0
    
    # Calculate hours from start_time and end_time
    duration = assignment.end_time - assignment.start_time
    hours = duration.total_seconds() / 3600
    emp_hours[emp_id] += hours

# Group by role for fairness analysis
role_hours = {}
for emp_id, hours in emp_hours.items():
    employee = EmployeeRepository.get_by_id(session, emp_id)
    role = employee.primary_role
    if role not in role_hours:
        role_hours[role] = {}
    role_hours[role][emp_id] = hours

for role, hours_dict in role_hours.items():
    print(f"\n{role} Fairness:")
    if len(hours_dict) > 1:
        min_hours = min(hours_dict.values())
        max_hours = max(hours_dict.values())
        avg_hours = sum(hours_dict.values()) / len(hours_dict)
        spread = max_hours - min_hours
        
        print(f"   Hours range: {min_hours:.1f}h - {max_hours:.1f}h")
        print(f"   Average: {avg_hours:.1f}h")
        print(f"   Spread: {spread:.1f}h")
        
        if spread <= 8:  # Within one shift
            print(f"   Good balance (spread <= 8h)")
        elif spread <= 16:  # Within two shifts
            print(f"   Moderate imbalance (spread <= 16h)")
        else:
            print(f"   Poor balance (spread > 16h)")
        
        # Show individual hours
        for emp_id, hours in sorted(hours_dict.items(), key=lambda x: x[1], reverse=True):
            employee = EmployeeRepository.get_by_id(session, emp_id)
            print(f"      {employee.first_name} {employee.last_name}: {hours:.1f}h")
    else:
        print(f"   Only one employee in this role")

# Show weekend coverage
print(f"\nWEEKEND COVERAGE:")
print("=" * 30)

weekend_shifts = [s for s in week_shifts if s.date.weekday() >= 5]  # Sat=5, Sun=6
for shift in weekend_shifts:
    day_name = shift.date.strftime("%A")
    shift_assignments = [a for a in week_assignments if a.shift_id == shift.shift_id]
    
    print(f"\n{shift.date} ({day_name}):")
    print(f"   {len(shift_assignments)} employees scheduled")
    
    # Group by role
    role_counts = {}
    for assign in shift_assignments:
        role = assign.role
        role_counts[role] = role_counts.get(role, 0) + 1
    
    for role, count in role_counts.items():
        print(f"   {role}: {count}")

print(f"\nDetailed logging complete!")


DETAILED SCHEDULING LOGS

Week 2025-W48 - 7 shifts, 32 assignments

Assignments by Role:

MANAGER ROLE (9 assignments):

   Max Hayes (MANAGER)
      Total: 40.0h across 5 shifts
      Skills: Coffee=None, Sandwich=None, Service=None, Speed=None
      2025-11-26 (Wednesday): 07:00-15:00 (8.0h)
      2025-11-28 (Friday): 07:00-15:00 (8.0h)
      2025-11-29 (Saturday): 07:00-15:00 (8.0h)
      2025-11-30 (Sunday): 07:00-15:00 (8.0h)
      2025-12-02 (Tuesday): 07:00-15:00 (8.0h)

   Mia Stone (MANAGER)
      Total: 32.0h across 4 shifts
      Skills: Coffee=None, Sandwich=None, Service=None, Speed=None
      2025-11-27 (Thursday): 07:00-15:00 (8.0h)
      2025-11-29 (Saturday): 07:00-15:00 (8.0h)
      2025-11-30 (Sunday): 07:00-15:00 (8.0h)
      2025-12-01 (Monday): 07:00-15:00 (8.0h)

SANDWICH ROLE (7 assignments):

   Sam Lee (SANDWICH)
      Total: 29.5h across 4 shifts
      Skills: Coffee=None, Sandwich=5.0, Service=None, Speed=None
      2025-11-26 (Wednesday): 05:00-12:00 (7.0h)

## Step 9: Export to CSV


In [18]:
# Check if we have assignments in the database
from scheduler.domain.repositories import AssignmentRepository
db_assignments = AssignmentRepository.get_by_week(session, WEEK_ID)
print(f"Assignments in database for {WEEK_ID}: {len(db_assignments)}")

if db_assignments:
    print("Found assignments in database!")
    
    # Show preview of assignments
    print(f"\nAssignment Preview:")
    for i, assign in enumerate(db_assignments[:5]):  # Show first 5
        employee = EmployeeRepository.get_by_id(session, assign.emp_id)
        shift = ShiftRepository.get_by_id(session, assign.shift_id)
        start_time = assign.start_time.strftime("%H:%M")
        end_time = assign.end_time.strftime("%H:%M")
        print(f"   {i+1}. {employee.first_name} {employee.last_name} ({assign.role}) - {shift.date} {start_time}-{end_time}")
    
    if len(db_assignments) > 5:
        print(f"   ... and {len(db_assignments) - 5} more assignments")
        
    # Try to export again
    print(f"\nAttempting to export assignments...")
    count = export_assignments_csv(session, OUTPUT_CSV, WEEK_ID)
    print(f"Exported {count} assignments to {OUTPUT_CSV}")
    
    # Show preview if file exists
    if Path(OUTPUT_CSV).exists():
        df = pd.read_csv(OUTPUT_CSV)
        print(f"\nCSV Preview:")
        print(df.head())
    else:
        print("Warning: CSV file was not created")
else:
    print("No assignments found in database")
    print("Schedule generation failed - check the error messages above")


Assignments in database for 2025-W48: 32
Found assignments in database!

Assignment Preview:
   1. Max Hayes (MANAGER) - 2025-11-29 07:00-15:00
   2. Mia Stone (MANAGER) - 2025-11-29 07:00-15:00
   3. Max Hayes (MANAGER) - 2025-11-30 07:00-15:00
   4. Mia Stone (MANAGER) - 2025-11-30 07:00-15:00
   5. Max Hayes (MANAGER) - 2025-11-26 07:00-15:00
   ... and 27 more assignments

Attempting to export assignments...
[INFO] Exported 32 assignments to schedule_present_2025_week_48.csv
Exported 32 assignments to schedule_present_2025_week_48.csv

CSV Preview:
   shift_id  emp_id           start_time             end_time
0      1087    1001  2025-11-29T07:00:00  2025-11-29T15:00:00
1      1087    1002  2025-11-29T07:00:00  2025-11-29T15:00:00
2      1088    1001  2025-11-30T07:00:00  2025-11-30T15:00:00
3      1088    1002  2025-11-30T07:00:00  2025-11-30T15:00:00
4      1084    1001  2025-11-26T07:00:00  2025-11-26T15:00:00


In [20]:
if assignments:
    print("Exporting to CSV...")
    count = export_assignments_csv(session, OUTPUT_CSV, WEEK_ID)
    print(f"Exported {count} assignments to {OUTPUT_CSV}")
    
    # Show preview if file exists
    if Path(OUTPUT_CSV).exists():
        df = pd.read_csv(OUTPUT_CSV)
        print(f"\nCSV Preview:")
        print(df.head())
    else:
        print("Warning: CSV file was not created")
else:
    print("No assignments to export")


session.close()



print("=" * 60)
print(f"Generated: {OUTPUT_CSV}")
print(f"Database: {DB_URL}")
print(f"Week: {WEEK_ID}")
print()



Exporting to CSV...
[INFO] Exported 32 assignments to schedule_present_2025_week_48.csv
Exported 32 assignments to schedule_present_2025_week_48.csv

CSV Preview:
   shift_id  emp_id           start_time             end_time
0      1087    1001  2025-11-29T07:00:00  2025-11-29T15:00:00
1      1087    1002  2025-11-29T07:00:00  2025-11-29T15:00:00
2      1088    1001  2025-11-30T07:00:00  2025-11-30T15:00:00
3      1088    1002  2025-11-30T07:00:00  2025-11-30T15:00:00
4      1084    1001  2025-11-26T07:00:00  2025-11-26T15:00:00
Generated: schedule_present_2025_week_48.csv
Database: sqlite:///scheduler_present.db
Week: 2025-W48

